# Team Operations: Reset, Stop, Resume and Abort

In [5]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_core.models import UserMessage
from dotenv import load_dotenv
load_dotenv()

model_client = OpenAIChatCompletionClient(model='gpt-4o-mini')

In [7]:
await model_client.create([UserMessage(content='Hi How are you', source='user')])

CreateResult(finish_reason='stop', content="Hello! I'm just a program, but I'm here and ready to help you. How can I assist you today?", usage=RequestUsage(prompt_tokens=12, completion_tokens=23), cached=False, logprobs=None, thought=None)

In [16]:
add_1_agent_first = AssistantAgent(
    name = 'add_1_agent_first',
    model_client=model_client,
    system_message='Add 1 to the number which is given to you as an input and give out result. Start with 0 if othing is provided'
)

add_1_agent_second = AssistantAgent(
    name = 'add_1_agent_second',
    model_client=model_client,
    system_message='Add 1 to the number provided and pass the result to the next agent'
)

add_1_agent_third = AssistantAgent(
    name = 'add_1_agent_third',
    model_client=model_client,
    system_message='Add 1 to the number provided and pass result to the next agent'
)

In [17]:
from autogen_agentchat.teams import RoundRobinGroupChat

team = RoundRobinGroupChat(
    participants=[add_1_agent_first, add_1_agent_second, add_1_agent_third],
    max_turns=3
)

In [18]:
from autogen_agentchat.ui import Console

await Console(team.run_stream(task = 'First number is 0'))


---------- TextMessage (user) ----------
First number is 0
---------- TextMessage (add_1_agent_first) ----------
1
---------- TextMessage (add_1_agent_second) ----------
2
---------- TextMessage (add_1_agent_third) ----------
3


TaskResult(messages=[TextMessage(id='a3a2cc69-5fc8-4c5b-b125-e00fca3d266c', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 35, 36, 827066, tzinfo=datetime.timezone.utc), content='First number is 0', type='TextMessage'), TextMessage(id='a5a40023-9c7f-4bfa-8e27-0712e5bf56e2', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=45, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 35, 37, 434015, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='63cea3fa-28e6-4f6e-92d3-0c88d23dace7', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=42, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 35, 37, 884029, tzinfo=datetime.timezone.utc), content='2', type='TextMessage'), TextMessage(id='ecfdc0ab-ee4c-473e-848b-4df44946d231', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=51, completion_tokens=1), metadata={},

### Resuming a Team

Teams are stateful and maintains the conversation history and content after each run, unless we reset the team.

We can resume a team to continue from where it left off bt calling the run() or run_stream() method without a new task.

In [19]:
await Console(team.run_stream(task = 'Continue'))

---------- TextMessage (user) ----------
Continue
---------- TextMessage (add_1_agent_first) ----------
4
---------- TextMessage (add_1_agent_second) ----------
5
---------- TextMessage (add_1_agent_third) ----------
6


TaskResult(messages=[TextMessage(id='d626c24a-26c8-497d-82e1-1bb34988f32b', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 39, 44, 377239, tzinfo=datetime.timezone.utc), content='Continue', type='TextMessage'), TextMessage(id='6ced021b-cb7d-4bbc-8232-50e1601d5524', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=77, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 39, 44, 771103, tzinfo=datetime.timezone.utc), content='4', type='TextMessage'), TextMessage(id='450d6e56-2647-4242-93f7-109c5b35aa6b', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=74, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 39, 45, 130865, tzinfo=datetime.timezone.utc), content='5', type='TextMessage'), TextMessage(id='eb46ecf1-524a-4a74-a0f2-10ded1c47b5b', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=82, completion_tokens=1), metadata={}, created_

## Reset Team

In [20]:
await team.reset()

In [21]:
await Console(team.run_stream())

---------- TextMessage (add_1_agent_first) ----------
0 + 1 = 1
---------- TextMessage (add_1_agent_second) ----------
1
---------- TextMessage (add_1_agent_third) ----------
2


TaskResult(messages=[TextMessage(id='53cfe20a-cc17-433e-ba08-4bfb3a4151ea', source='add_1_agent_first', models_usage=RequestUsage(prompt_tokens=35, completion_tokens=7), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 44, 5, 56491, tzinfo=datetime.timezone.utc), content='0 + 1 = 1', type='TextMessage'), TextMessage(id='7cdc774a-29c0-46b5-b16f-9b2f47fb4b61', source='add_1_agent_second', models_usage=RequestUsage(prompt_tokens=38, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 44, 5, 455729, tzinfo=datetime.timezone.utc), content='1', type='TextMessage'), TextMessage(id='31bc6da4-c080-4dac-adac-53506e10f32a', source='add_1_agent_third', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=1), metadata={}, created_at=datetime.datetime(2025, 8, 8, 3, 44, 5, 859163, tzinfo=datetime.timezone.utc), content='2', type='TextMessage')], stop_reason='Maximum number of turns 3 reached.')